In [1]:
import pyspch
import pyspch.nn

D:\ProgramFiles\miniconda3\envs\pyspch_edit\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


## Dummy setup

In [2]:
import random
import torch
import numpy as np

In [63]:
## Dataset

# params
# params
n = 10
indim, timedim = 50, 1000
outdim = 2
label_set = [0, 1]

# inputs & labels
corpus_list = [str(i) for i in range(n)]
input_list = [np.random.randn(indim, timedim) for _ in range(n)]
mean_input_list = [np.mean(input, axis=0) for input in input_list]
label_list = mean_input_list
for label, input in zip(label_list, mean_input_list):
    label[input > 0] = label_set[1]
    label[input < 0] = label_set[0]
    
# dataset
spchds = pyspch.nn.SpchDataset(corpus_list, input_list, label_list)

# sampler
lengths = [item.shape[-1] for item in input_list]
spchds.set_sampler(lengths, {'N': 5, 'stride': 1, 'mode': 'flatten1d'})

# target encoding 
# lab2idx = {k: i for i, k in enumerate(label_set)}
# spchds.encode_target(lab2idx)

# format
spchds.to_tensor()

# to device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
spchds.to_device(device)

In [65]:
# example
x, y = spchds.__getitem__(5)
print(x.shape)
print(y.shape)
print(x.dtype)
print(y.dtype)

torch.Size([550])
torch.Size([])
torch.float32
torch.int64


In [66]:
## Model + training setup

# model
in_dim = x.shape[0]
out_dim = 2
hidden_dims = [10, 10]
model = pyspch.nn.FFDNN(in_dim, out_dim, hidden_dims)
model.to(device)

# criterion & optimizer
lr = 0.0001
weight_decay = 0
criterion = torch.nn.CrossEntropyLoss() # applies softmax()
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

# dataloader
batch_size = 64
shuffle = True
train_dl = torch.utils.data.DataLoader(spchds, batch_size=batch_size, shuffle=shuffle)

In [67]:
## Training

# train model
n_epochs = 50
valid_dl = None
patience = 5
every = 5 
current_epoch = 0
tr_loss, val_loss, epoch = pyspch.nn.train(model, train_dl, criterion, optimizer, 
    clip_args=None, scheduler=None,
    current_epoch=current_epoch, n_epochs=n_epochs, 
    valid_dl=valid_dl, patience=patience, every=every)

Epoch 0 -- av. train loss per mini-batch 0.70
Epoch 5 -- av. train loss per mini-batch 0.65
Epoch 10 -- av. train loss per mini-batch 0.55
Epoch 15 -- av. train loss per mini-batch 0.41
Epoch 20 -- av. train loss per mini-batch 0.29
Epoch 25 -- av. train loss per mini-batch 0.20
Epoch 30 -- av. train loss per mini-batch 0.15
Epoch 35 -- av. train loss per mini-batch 0.12
Epoch 40 -- av. train loss per mini-batch 0.09
Epoch 45 -- av. train loss per mini-batch 0.08
